Churn Prediction

In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [19]:
data = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv"

In [20]:
!wget $data

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [22]:
len(df)

7043

Initial Data Preparation

In [23]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [24]:
# We transpose to see all columns
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [25]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [26]:
# We prepare the data

df.columns = df.columns.str.lower().str.replace(" ", "_")
df.columns
string_columns = list(df.dtypes[df.dtypes == "object"].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(" ", "_")

In [27]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [28]:
df.dtypes
# we can see that the type of total charges is an object. It should be a float/int

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [29]:
df["totalcharges"]

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: totalcharges, Length: 7043, dtype: object

In [30]:
# we have to convert all values to numbers
# pd.to_numeric(df["totalcharges"])
# this shows the position at which the conversion cannot be carried out
# It also shows what kind of data causes the issue. In our case "_". It can either be caused by an 
# underscore or a space.
#  we will convert/coerce the error to Nan, and then replace the Nan with 0


df["totalcharges"] = pd.to_numeric(df["totalcharges"], errors="coerce")
df[df.totalcharges.isnull()][["customerid", "totalcharges"]]
df["totalcharges"].iloc[488]
df["totalcharges"] = df["totalcharges"].fillna(0)
df["totalcharges"].iloc[488]
df["totalcharges"][df["totalcharges"] == 0]
df["totalcharges"][df["totalcharges"] == 0].value_counts()
df[df.totalcharges == 0][["customerid", "totalcharges"]]

,customerid,totalcharges
488,4472-lvygi,0.0
753,3115-czmzd,0.0
936,5709-lvoeq,0.0
1082,4367-nuyao,0.0
1340,1371-dwpaz,0.0
3331,7644-omvmy,0.0
3826,3213-vvolg,0.0
4380,2520-sgtta,0.0
5218,2923-arzlg,0.0
6670,4075-wkniu,0.0


In [31]:
# We check our churn variable and convert it no numbers, not yes/no
# (yes = true = 1) and (no = 0 = false)
df.churn
df.churn.head(5)
(df.churn == "yes").astype(int).head()
df.churn = (df.churn == "yes").astype(int)

In [32]:
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

Setting up Validation Framework using Scikit

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
# to see the documentation
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [35]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
len(df_full_train), len(df_test), len(df)

(5634, 1409, 7043)

In [36]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)
len(df_train), len(df_val), len(df_test), len(df_full_train), len(df)

(4225, 1409, 1409, 5634, 7043)

In [37]:
# We reset the index and extract the y values

df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [38]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [39]:
del df_train["churn"]
del df_val["churn"]
del df_test["churn"]

# Exploratory Data Analysis
- Check missing values
- Look at the target values (churn)
- Look at numerical and Categorical Variables

In [40]:
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [41]:
# value_counts counts the number of all values in the column
# This shows us the number of churning users(1) and non-churning users(0)
df_full_train.churn.value_counts()
df_full_train.churn.value_counts(normalize=True)        #This shows percentage
# The "1" shows us the churn rate

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [42]:
# We can also compute the churn rate using the mean
df_full_train.churn.mean()
# This gives the same value because the mean is calculated using "1s"

np.float64(0.26996805111821087)

In [43]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 3)
# 27% of users are churning
# This gives an understanding of our target variable

np.float64(0.27)

In [44]:
# We consider our feature Variables - categorical or numerical
df_full_train.nunique()
# This shows you the categorical variables even though they might be numerical variables
# df_full_train.dtypes

customerid          5634
gender                 2
seniorcitizen          2
partner                2
dependents             2
tenure                73
phoneservice           2
multiplelines          3
internetservice        3
onlinesecurity         3
onlinebackup           3
deviceprotection       3
techsupport            3
streamingtv            3
streamingmovies        3
contract               3
paperlessbilling       2
paymentmethod          4
monthlycharges      1494
totalcharges        5291
churn                  2
dtype: int64

In [45]:
# We group the categorical separate from numerical
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [46]:
df_full_train[categorical]
df_full_train[categorical].nunique()
# some of the categorical variables are binary

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

# Feature Importance

In [48]:
female_mean = df_full_train[df_full_train.gender == 'female'].churn.mean()
print('gender == female:', round(female_mean, 3))

male_mean = df_full_train[df_full_train.gender == 'male'].churn.mean()
print('gender == male:  ', round(male_mean, 3))

gender == female: 0.277
gender == male:   0.263


In [49]:
female_mean / global_churn_rate


np.float64(1.0253955354648652)

In [50]:
male_mean / global_churn_rate

np.float64(0.9749802969838747)

In [ ]:
partner_yes = df_full_train[df_full_train.partner == 'yes'].churn.mean()
print('partner == yes:', round(partner_yes, 3))

partner_no = df_full_train[df_full_train.partner == 'no'].churn.mean()
print('partner == no :', round(partner_no, 3))

partner == yes: 0.205
partner == no : 0.33


In [52]:
partner_yes / global_churn_rate

np.float64(0.7594724924338315)

In [53]:
partner_no / global_churn_rate

np.float64(1.2216593879412643)

In [54]:
df_group = df_full_train.groupby(by='gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_churn_rate
df_group['risk'] = df_group['mean'] / global_churn_rate
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [55]:
from IPython.display import display

In [56]:
for col in categorical:
    df_group = df_full_train.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_churn_rate
    df_group['risk'] = df_group['mean'] / global_churn_rate
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


# Feature Importance: Mutual Information 

In [73]:
# Mutual information is a concept that tells us how much we can learn about one variable if we know the value of another.
# How do we know what variable is more important in churn prediction
# Is gender more important than contract, partner, etc
from sklearn.metrics import mutual_info_score

In [74]:
mutual_info_score(df_full_train.churn, df_full_train.contract)
# This tells us how much we learn about churn by observing the contract variable

0.0983203874041556

In [75]:
mutual_info_score(df_full_train.churn, df_full_train.gender)
# This tells us that gender is not a big consideration for churn

0.0001174846211139946

In [76]:
mutual_info_score(df_full_train.churn, df_full_train.partner)
# partner is more important than gender

0.009967689095399745

In [81]:
# We apply to all variables to know which of them is more important
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [86]:
series1 = df_full_train[categorical].apply(mutual_info_churn_score)
# This shows how important each column is to churning
series1.sort_values(ascending=False)
# we can see that contract, onlinesecurity, techsupport, down to paymentmethod are very important for predicting churn rate of customers

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64